This is an attempt to recreate the parameter estimation [example](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html) from James Rawlings book on [Reactor Design](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/) using Pyomo [parmest](https://pyomo.readthedocs.io/en/stable/contributed_packages/parmest/driver.html).

In [1]:
# Import libraries
from pyomo.environ import *
from pyomo.dae import *
import pyomo.contrib.parmest.parmest as parmest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

This example has a series reaction $A \rightarrow B \rightarrow C$. The dataset consists of measures concentrations of A, B and C over time. The goal is to estimate the rate constants $k_1$ and $k_2$ for the two reactions.

In [2]:
data_df = pd.read_csv("ABC_data.csv")
data_df.head()

,t,ca,cb,cc
0,0.000,0.957,-0.031,-0.015
1,0.263,0.557,0.330,0.044
2,0.526,0.342,0.512,0.156
3,0.789,0.224,0.499,0.310
4,1.053,0.123,0.428,0.454


In [3]:
# Convert data to a list of dictionaries
data = [{'ca_meas': {k:v for (k, v) in zip(data_df.t, data_df.ca)},
    'cb_meas': {k:v for (k, v) in zip(data_df.t, data_df.cb)},
    'cc_meas': {k:v for (k, v) in zip(data_df.t, data_df.cc)} }]

In [4]:
#
# Define the model 
#
def ABC_model(data):
    
    ca_meas = data['ca_meas']
    cb_meas = data['cb_meas']
    cc_meas = data['cc_meas']
    
    meas_t = list(ca_meas.keys())
       
    ca0 = 1.0
    cb0 = 0.0
    cc0 = 0.0
        
    m = ConcreteModel()
    
    m.k1 = Var(initialize = 0.5, bounds = (1e-4, 10))
    m.k2 = Var(initialize = 3.0, bounds = (1e-4, 10))
    
    m.time = ContinuousSet(bounds = (0.0, 5.0), initialize = meas_t)
    m.ca = Var(m.time, initialize = ca0, bounds = (0, ca0))
    m.cb = Var(m.time, initialize = cb0, bounds = (0, ca0))
    m.cc = Var(m.time, initialize = cc0, bounds = (0, ca0))
    
    m.dca = DerivativeVar(m.ca, wrt = m.time)
    m.dcb = DerivativeVar(m.cb, wrt = m.time)
    m.dcc = DerivativeVar(m.cc, wrt = m.time)
    
    def _dcarate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dca[t] == -m.k1 * m.ca[t]
    m.dcarate = Constraint(m.time, rule = _dcarate)
    
    def _dcbrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcb[t] == m.k1 * m.ca[t] - m.k2 * m.cb[t]
    m.dcbrate = Constraint(m.time, rule = _dcbrate)
    
    def _dccrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcc[t] == m.k2 * m.cb[t]
    m.dccrate = Constraint(m.time, rule = _dccrate)
    
    def _initcon(m):
        yield m.ca[m.time.first()] == ca0
        yield m.cb[m.time.first()] == cb0
        yield m.cc[m.time.first()] == cc0
    m.initcon = ConstraintList(rule = _initcon)
    
    def ComputeFirstStageCost_rule(m):
        return 0
    m.FirstStageCost = Expression(rule=ComputeFirstStageCost_rule)

    def ComputeSecondStageCost_rule(m):
        return sum((m.ca[t] - ca_meas[t]) ** 2 + (m.cb[t] - cb_meas[t]) ** 2 
                   + (m.cc[t] - cc_meas[t]) ** 2 for t in meas_t) 
    m.SecondStageCost = Expression(rule=ComputeSecondStageCost_rule)

    
    def total_cost_rule(model):
        return model.FirstStageCost + model.SecondStageCost
    m.Total_Cost_Objective = Objective(rule=total_cost_rule, sense=minimize)
    
    disc = TransformationFactory('dae.collocation')
    disc.apply_to(m, nfe=20, ncp=4)
    
    return m

In [5]:
# parameters to be estimated
theta_names = ['k1', 'k2']

In [6]:
pest = parmest.Estimator(ABC_model, data, theta_names, tee = True)

In [7]:
obj, theta = pest.theta_est()

Ipopt 3.11.1: max_iter=6000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NOTE: You are using Ipopt by default with the MUMPS linear solver.
      Other linear solvers might be more efficient (see Ipopt documentation).


This is Ipopt version 3.11.1, running with linear solver mumps.

Number of nonzeros in equality constraint Jacobian...:     2327
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      220

Total number of variables............................:      487
                     variables with only lower bounds:        0
                variables with lower and upper bou

 120  2.5482118e+01 3.73e-08 6.62e-01  -2.5 4.04e-04   3.2 1.00e+00 1.00e+00f  1
 121  2.5477142e+01 5.24e-09 6.61e-01  -2.5 1.51e-04   3.6 1.00e+00 1.00e+00f  1
 122  2.5462229e+01 4.71e-08 6.61e-01  -2.5 4.54e-04   3.2 1.00e+00 1.00e+00f  1
 123  2.5456639e+01 6.62e-09 6.60e-01  -2.5 1.70e-04   3.6 1.00e+00 1.00e+00f  1
 124  2.5439889e+01 5.94e-08 6.60e-01  -2.5 5.10e-04   3.1 1.00e+00 1.00e+00f  1
 125  2.5433610e+01 8.36e-09 6.60e-01  -2.5 1.91e-04   3.5 1.00e+00 1.00e+00f  1
 126  2.5414801e+01 7.51e-08 6.59e-01  -2.5 5.73e-04   3.1 1.00e+00 1.00e+00f  1
 127  2.5407751e+01 1.06e-08 6.58e-01  -2.5 2.15e-04   3.5 1.00e+00 1.00e+00f  1
 128  2.5386634e+01 9.48e-08 6.58e-01  -2.5 6.44e-04   3.0 1.00e+00 1.00e+00f  1
 129  2.5378720e+01 1.33e-08 6.57e-01  -2.5 2.41e-04   3.4 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 130  2.5355017e+01 1.20e-07 6.57e-01  -2.5 7.23e-04   3.0 1.00e+00 1.00e+00f  1
 131  2.5346134e+01 1.68e-08

 245  2.4283818e+01 4.01e-08 6.18e-01  -2.5 4.10e-04   3.2 1.00e+00 1.00e+00f  1
 246  2.4278832e+01 5.64e-09 6.18e-01  -2.5 1.54e-04   3.6 1.00e+00 1.00e+00f  1
 247  2.4263887e+01 5.07e-08 6.18e-01  -2.5 4.61e-04   3.1 1.00e+00 1.00e+00f  1
 248  2.4258284e+01 7.13e-09 6.17e-01  -2.5 1.73e-04   3.6 1.00e+00 1.00e+00f  1
 249  2.4241494e+01 6.41e-08 6.17e-01  -2.5 5.18e-04   3.1 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 250  2.4235201e+01 9.01e-09 6.17e-01  -2.5 1.94e-04   3.5 1.00e+00 1.00e+00f  1
 251  2.4216342e+01 8.09e-08 6.16e-01  -2.5 5.82e-04   3.0 1.00e+00 1.00e+00f  1
 252  2.4209273e+01 1.14e-08 6.16e-01  -2.5 2.18e-04   3.5 1.00e+00 1.00e+00f  1
 253  2.4206623e+01 1.60e-09 6.16e-01  -2.5 8.18e-05   3.9 1.00e+00 1.00e+00f  1
 254  2.4198676e+01 1.44e-08 6.16e-01  -2.5 2.45e-04   3.4 1.00e+00 1.00e+00f  1
 255  2.4195697e+01 2.02e-09 6.15e-01  -2.5 9.20e-05   3.8 1.00e+00 1.00e+00f  1
 256  2.4186763e+01 1.82e-08

 372  2.3219740e+01 5.62e-08 5.87e-01  -2.5 4.76e-04   3.1 1.00e+00 1.00e+00f  1
 373  2.3214026e+01 7.91e-09 5.86e-01  -2.5 1.78e-04   3.5 1.00e+00 1.00e+00f  1
 374  2.3196901e+01 7.10e-08 5.86e-01  -2.5 5.35e-04   3.0 1.00e+00 1.00e+00f  1
 375  2.3190481e+01 9.99e-09 5.86e-01  -2.5 2.01e-04   3.5 1.00e+00 1.00e+00f  1
 376  2.3188074e+01 1.40e-09 5.86e-01  -2.5 7.52e-05   3.9 1.00e+00 1.00e+00f  1
 377  2.3180857e+01 1.26e-08 5.86e-01  -2.5 2.26e-04   3.4 1.00e+00 1.00e+00f  1
 378  2.3178151e+01 1.78e-09 5.85e-01  -2.5 8.46e-05   3.8 1.00e+00 1.00e+00f  1
 379  2.3170037e+01 1.60e-08 5.85e-01  -2.5 2.54e-04   3.4 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380  2.3166994e+01 2.25e-09 5.85e-01  -2.5 9.51e-05   3.8 1.00e+00 1.00e+00f  1
 381  2.3157872e+01 2.02e-08 5.85e-01  -2.5 2.85e-04   3.3 1.00e+00 1.00e+00f  1
 382  2.3154452e+01 2.84e-09 5.85e-01  -2.5 1.07e-04   3.7 1.00e+00 1.00e+00f  1
 383  2.3144198e+01 2.56e-08

 498  2.2291583e+01 8.89e-09 5.62e-01  -2.5 1.86e-04   3.5 1.00e+00 1.00e+00f  1
 499  2.2289377e+01 1.25e-09 5.62e-01  -2.5 6.97e-05   3.9 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500  2.2282759e+01 1.12e-08 5.62e-01  -2.5 2.09e-04   3.4 1.00e+00 1.00e+00f  1
 501  2.2280278e+01 1.58e-09 5.61e-01  -2.5 7.84e-05   3.9 1.00e+00 1.00e+00f  1
 502  2.2272838e+01 1.42e-08 5.61e-01  -2.5 2.35e-04   3.4 1.00e+00 1.00e+00f  1
 503  2.2270048e+01 2.00e-09 5.61e-01  -2.5 8.81e-05   3.8 1.00e+00 1.00e+00f  1
 504  2.2261683e+01 1.80e-08 5.61e-01  -2.5 2.64e-04   3.3 1.00e+00 1.00e+00f  1
 505  2.2258546e+01 2.53e-09 5.61e-01  -2.5 9.91e-05   3.8 1.00e+00 1.00e+00f  1
 506  2.2249142e+01 2.28e-08 5.61e-01  -2.5 2.97e-04   3.3 1.00e+00 1.00e+00f  1
 507  2.2245617e+01 3.20e-09 5.61e-01  -2.5 1.11e-04   3.7 1.00e+00 1.00e+00f  1
 508  2.2235046e+01 2.88e-08 5.60e-01  -2.5 3.34e-04   3.2 1.00e+00 1.00e+00f  1
 509  2.2231082e+01 4.05e-09

...

5407  1.0625198e+01 6.27e-10 2.78e-01  -3.8 3.44e-05   3.9 1.00e+00 1.00e+00f  1
5408  1.0624848e+01 8.81e-11 2.78e-01  -3.8 1.29e-05   4.3 1.00e+00 1.00e+00h  1
5409  1.0623801e+01 7.93e-10 2.78e-01  -3.8 3.87e-05   3.9 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5410  1.0623408e+01 1.11e-10 2.78e-01  -3.8 1.45e-05   4.3 1.00e+00 1.00e+00h  1
5411  1.0622231e+01 1.00e-09 2.78e-01  -3.8 4.36e-05   3.8 1.00e+00 1.00e+00f  1
5412  1.0621789e+01 1.41e-10 2.78e-01  -3.8 1.63e-05   4.2 1.00e+00 1.00e+00h  1
5413  1.0620464e+01 1.27e-09 2.78e-01  -3.8 4.90e-05   3.8 1.00e+00 1.00e+00f  1
5414  1.0619967e+01 1.79e-10 2.78e-01  -3.8 1.84e-05   4.2 1.00e+00 1.00e+00h  1
5415  1.0618477e+01 1.61e-09 2.78e-01  -3.8 5.51e-05   3.7 1.00e+00 1.00e+00f  1
5416  1.0617918e+01 2.26e-10 2.78e-01  -3.8 2.07e-05   4.1 1.00e+00 1.00e+00h  1
5417  1.0616242e+01 2.03e-09 2.78e-01  -3.8 6.20e-05   3.7 1.00e+00 1.00e+00f  1
5418  1.0615613e+01 2.86e-10

5537  1.0503924e+01 1.65e-10 2.75e-01  -3.8 1.76e-05   4.2 1.00e+00 1.00e+00h  1
5538  1.0502501e+01 1.49e-09 2.75e-01  -3.8 5.27e-05   3.7 1.00e+00 1.00e+00f  1
5539  1.0501967e+01 2.09e-10 2.75e-01  -3.8 1.98e-05   4.1 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5540  1.0500367e+01 1.88e-09 2.75e-01  -3.8 5.93e-05   3.7 1.00e+00 1.00e+00f  1
5541  1.0499767e+01 2.65e-10 2.75e-01  -3.8 2.22e-05   4.1 1.00e+00 1.00e+00h  1
5542  1.0497966e+01 2.38e-09 2.75e-01  -3.8 6.67e-05   3.6 1.00e+00 1.00e+00f  1
5543  1.0497291e+01 3.35e-10 2.75e-01  -3.8 2.50e-05   4.0 1.00e+00 1.00e+00h  1
5544  1.0495266e+01 3.01e-09 2.75e-01  -3.8 7.50e-05   3.6 1.00e+00 1.00e+00f  1
5545  1.0494507e+01 4.24e-10 2.75e-01  -3.8 2.81e-05   4.0 1.00e+00 1.00e+00h  1
5546  1.0494222e+01 5.96e-11 2.75e-01  -3.8 1.05e-05   4.4 1.00e+00 1.00e+00h  1
5547  1.0493368e+01 5.36e-10 2.75e-01  -3.8 3.16e-05   3.9 1.00e+00 1.00e+00f  1
5548  1.0493048e+01 7.54e-11

5668  1.0380300e+01 3.92e-10 2.72e-01  -3.8 2.69e-05   4.0 1.00e+00 1.00e+00h  1
5669  1.0380028e+01 5.51e-11 2.72e-01  -3.8 1.01e-05   4.4 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5670  1.0379213e+01 4.96e-10 2.72e-01  -3.8 3.03e-05   4.0 1.00e+00 1.00e+00f  1
5671  1.0378907e+01 6.98e-11 2.72e-01  -3.8 1.13e-05   4.4 1.00e+00 1.00e+00h  1
5672  1.0377990e+01 6.28e-10 2.72e-01  -3.8 3.40e-05   3.9 1.00e+00 1.00e+00f  1
5673  1.0377646e+01 8.83e-11 2.72e-01  -3.8 1.28e-05   4.3 1.00e+00 1.00e+00h  1
5674  1.0376614e+01 7.94e-10 2.72e-01  -3.8 3.83e-05   3.9 1.00e+00 1.00e+00f  1
5675  1.0376227e+01 1.12e-10 2.72e-01  -3.8 1.44e-05   4.3 1.00e+00 1.00e+00h  1
5676  1.0375066e+01 1.01e-09 2.72e-01  -3.8 4.31e-05   3.8 1.00e+00 1.00e+00f  1
5677  1.0374631e+01 1.41e-10 2.72e-01  -3.8 1.62e-05   4.2 1.00e+00 1.00e+00h  1
5678  1.0373325e+01 1.27e-09 2.72e-01  -3.8 4.85e-05   3.7 1.00e+00 1.00e+00f  1
5679  1.0372835e+01 1.79e-10

5796  1.0266863e+01 8.17e-11 2.70e-01  -3.8 1.22e-05   4.3 1.00e+00 1.00e+00h  1
5797  1.0265878e+01 7.35e-10 2.70e-01  -3.8 3.66e-05   3.9 1.00e+00 1.00e+00f  1
5798  1.0265508e+01 1.03e-10 2.70e-01  -3.8 1.37e-05   4.3 1.00e+00 1.00e+00h  1
5799  1.0264399e+01 9.30e-10 2.70e-01  -3.8 4.12e-05   3.8 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5800  1.0263984e+01 1.31e-10 2.70e-01  -3.8 1.55e-05   4.2 1.00e+00 1.00e+00h  1
5801  1.0262736e+01 1.18e-09 2.69e-01  -3.8 4.64e-05   3.8 1.00e+00 1.00e+00f  1
5802  1.0262269e+01 1.66e-10 2.69e-01  -3.8 1.74e-05   4.2 1.00e+00 1.00e+00h  1
5803  1.0260866e+01 1.49e-09 2.69e-01  -3.8 5.21e-05   3.7 1.00e+00 1.00e+00f  1
5804  1.0260340e+01 2.10e-10 2.69e-01  -3.8 1.96e-05   4.1 1.00e+00 1.00e+00h  1
5805  1.0258763e+01 1.89e-09 2.69e-01  -3.8 5.86e-05   3.7 1.00e+00 1.00e+00f  1
5806  1.0258171e+01 2.65e-10 2.69e-01  -3.8 2.20e-05   4.1 1.00e+00 1.00e+00h  1
5807  1.0256396e+01 2.39e-09

5924  1.0153482e+01 1.09e-09 2.67e-01  -3.8 4.44e-05   3.8 1.00e+00 1.00e+00f  1
5925  1.0153035e+01 1.53e-10 2.67e-01  -3.8 1.66e-05   4.2 1.00e+00 1.00e+00h  1
5926  1.0151695e+01 1.38e-09 2.67e-01  -3.8 4.99e-05   3.7 1.00e+00 1.00e+00f  1
5927  1.0151193e+01 1.94e-10 2.67e-01  -3.8 1.87e-05   4.2 1.00e+00 1.00e+00h  1
5928  1.0149686e+01 1.75e-09 2.67e-01  -3.8 5.61e-05   3.7 1.00e+00 1.00e+00f  1
5929  1.0149121e+01 2.45e-10 2.67e-01  -3.8 2.10e-05   4.1 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5930  1.0147426e+01 2.21e-09 2.67e-01  -3.8 6.31e-05   3.6 1.00e+00 1.00e+00f  1
5931  1.0146790e+01 3.11e-10 2.67e-01  -3.8 2.37e-05   4.1 1.00e+00 1.00e+00h  1
5932  1.0144884e+01 2.79e-09 2.67e-01  -3.8 7.10e-05   3.6 1.00e+00 1.00e+00f  1
5933  1.0144169e+01 3.93e-10 2.67e-01  -3.8 2.66e-05   4.0 1.00e+00 1.00e+00h  1
5934  1.0143901e+01 5.53e-11 2.67e-01  -3.8 9.98e-06   4.4 1.00e+00 1.00e+00h  1
5935  1.0143097e+01 4.97e-10